In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# import datetime
# import pandas as pd
# from time import time
# # from autograd import grad
# # import autograd.numpy as np
# import numpy as np
# from numba import njit
# from scipy.optimize import minimize, fsolve

# # CPMP's logloss from https://www.kaggle.com/c/lish-moa/discussion/183010
# def log_loss_numpy(y_pred):
#     y_true_ravel = np.asarray(y_true).ravel()
#     y_pred = np.asarray(y_pred).ravel()
#     y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
#     loss = np.where(y_true_ravel == 1, - np.log(y_pred), - np.log(1 - y_pred))
#     return loss.mean()

# def func_numpy_metric(weights):
#     oof_blend = np.tensordot(weights, oof, axes = ((0), (0)))
#     return log_loss_numpy(oof_blend)

# def grad_func(weights):
#     oof_clip = np.clip(oof, 1e-15, 1 - 1e-15)
#     gradients = np.zeros(oof.shape[0])
#     for i in range(oof.shape[0]):
#         a, b, c = y_true, oof_clip[i], np.zeros((oof.shape[1], oof.shape[2]))
#         for j in range(oof.shape[0]):
#             if j != i:
#                 c += weights[j] * oof_clip[j]
#         gradients[i] = -np.mean((-a*b+(b**2)*weights[i]+b*c)/((b**2)*(weights[i]**2)+2*b*c*weights[i]-b*weights[i]+(c**2)-c))
#     return gradients

# @njit
# def grad_func_jit(weights):
#     oof_clip = np.minimum(1 - 1e-15, np.maximum(oof, 1e-15))
#     gradients = np.zeros(oof.shape[0])
#     for i in range(oof.shape[0]):
#         a, b, c = y_true, oof_clip[i], np.zeros((oof.shape[1], oof.shape[2]))
#         for j in range(oof.shape[0]):
#             if j != i:
#                 c += weights[j] * oof_clip[j]
#         gradients[i] = -np.mean((-a*b+(b**2)*weights[i]+b*c)/((b**2)*(weights[i]**2)+2*b*c*weights[i]-b*weights[i]+(c**2)-c))
#     return gradients

In [ ]:
# y_true = pd.read_csv('../input/lish-moa/train_targets_scored.csv', index_col = 'sig_id').values

# oof_dict = {'Model 1': '../input/nn-transfer-learning-oof/oof_nn_transfer_all.npy',
#             'Model 2': '../input/tabnet-oof-seed3-fold7-112220/oof_TabNet_all.npy',
#             'Resnet': '../input/prediction/resnet_oof.npy',
#             'Multi_heads': '../input/jp-multi-heads/multi_heads_oof.npy'
#            }

# oof = np.zeros((len(oof_dict), y_true.shape[0], y_true.shape[1]))
# for i in range(oof.shape[0]):
#     #print(list(oof_dict.values())[i])
#     oof[i] = np.load(list(oof_dict.values())[i])

In [ ]:
# %%time

# log_loss_scores = {}
# for n, key in enumerate(oof_dict.keys()):
#     score_oof = log_loss_numpy(oof[n])
#     log_loss_scores[key] = score_oof
#     print(f'{key} CV:\t', score_oof)
# print('-' * 50)

In [ ]:
# test_weights = np.array([1 / oof.shape[0]] * oof.shape[0])

In [ ]:
# tol = 1e-10
# init_guess = [1 / oof.shape[0]] * oof.shape[0]
# bnds = [(0, 1) for _ in range(oof.shape[0])]
# cons = {'type': 'eq', 
#         'fun': lambda x: np.sum(x) - 1, 
#         'jac': lambda x: [1] * len(x)}

# print('Inital Blend OOF:', func_numpy_metric(init_guess))
# start_time = time()
# res_scipy = minimize(fun = func_numpy_metric, 
#                      x0 = init_guess, 
#                      method = 'SLSQP', 
#                      jac = grad_func_jit, # grad_func 
#                      bounds = bnds, 
#                      constraints = cons, 
#                      tol = tol)
# print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] Optimised Blend OOF:', res_scipy.fun)
# print('Optimised Weights:', res_scipy.x)

In [ ]:
# print('Check the sum of all weights:', np.sum(res_scipy.x))
# if np.sum(res_scipy.x) - 1 <= tol:
#     print('Great! The sum of all weights equals to 1!')
# else:
#     print('Manual adjustion is needed to modify the weights.')

In [ ]:
sub_nn_transfer = pd.read_csv("../input/subcsv/submission_nn_transfer.csv")
submission_TabNet = pd.read_csv("../input/subcsv/submission_TabNet.csv")
sub_rs = pd.read_csv("../input/subcsv/submission_rs.csv")
sub_mh = pd.read_csv("../input/subcsv/submission_mh.csv")

In [ ]:
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')

In [ ]:
sub = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
# sub.iloc[:,1:] = np.clip(res_scipy.x[0]*sub_nn_transfer.iloc[:, 1:].values + \
#                   res_scipy.x[1]*submission_TabNet.iloc[:,1:].values + \
#                   res_scipy.x[2]*sub_rs.iloc[:,1:].values + \
#                   res_scipy.x[3]*sub_mh.iloc[:,1:].values, 0.000005,0.999995)

sub.iloc[:,1:] = np.clip(0.41689*sub_nn_transfer.iloc[:, 1:].values + \
                  0.25914*submission_TabNet.iloc[:,1:].values + \
                  0.08506*sub_rs.iloc[:,1:].values + \
                  0.23892*sub_mh.iloc[:,1:].values, 0.000005,0.999995)

# sub.iloc[:,1:] = (0.4*sub_nn_transfer.iloc[:, 1:].values + \
#                   0.2*submission_TabNet.iloc[:,1:].values + \
#                   0.4*sub_rs.iloc[:,1:].values)

# Set ctl_vehicle to 0
sub.iloc[test_features['cp_type'] == 'ctl_vehicle',1:] = 0

# Save Submission
sub.to_csv('submission.csv', index=False)

In [ ]:
sub

In [ ]:
!ls 